In [27]:
import os
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np

from pathlib import Path
import pandas as pd
import nibabel as nib

import torch
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn.functional as F

import sklearn
import sklearn.metrics

import gradio as gr

In [19]:
dataset_dir = Path(os.getcwd()) / "Datasets" / "ADNI" / "ADNI1"
print(dataset_dir)

imgs_dir = dataset_dir / "Images"
masks_dir = dataset_dir / "SegmentationMasks"
store_dir = dataset_dir / "InputImages"
input_dir = store_dir

print(imgs_dir)
print(masks_dir)

csv_files = dataset_dir.glob("**/*.csv")
for csv_file in csv_files:
    csv_data = csv_file
    
print(csv_data)
df = pd.read_csv(csv_data)

d:\Facultate\anul_3\ELL\Code\Datasets\ADNI\ADNI1
d:\Facultate\anul_3\ELL\Code\Datasets\ADNI\ADNI1\Images
d:\Facultate\anul_3\ELL\Code\Datasets\ADNI\ADNI1\SegmentationMasks
d:\Facultate\anul_3\ELL\Code\Datasets\ADNI\ADNI1\ADNI1_T1_Imgs_CN-MCI-AD_5_18_2024.csv


In [24]:
df.where(df["Group"] == "AD").notna()

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2715,False,False,False,False,False,False,False,False,False,False,False,False
2716,False,False,False,False,False,False,False,False,False,False,False,False
2717,False,False,False,False,False,False,False,False,False,False,False,False
2718,False,False,False,False,False,False,False,False,False,False,False,False


tensor([[-18.3803,  13.6432]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-18.3952,  13.7174]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-18.3952,  13.7174]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-18.3952,  13.7174]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-16.1119,  12.3006]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [2]:
import importlib
import models.ResNet as RN
_ = importlib.reload(RN)

In [36]:
label_dict = {
    "CN": 0,
    "AD": 1,
}

num_classes = len(label_dict.keys())
save_path = "model_resnet_best.pth"

torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RN.ResNet18(device, num_classes)

model.conv1 = torch.nn.Conv2d(1, model.conv1.out_channels, kernel_size=model.conv1.kernel_size, stride=model.conv1.stride, padding=model.conv1.padding, bias=model.conv1.bias)
model.fc2 = torch.nn.Linear(in_features=model.fc2.in_features, out_features=2, bias=True)

if os.path.exists(save_path):
    print(f"found saved state at: {save_path}")
    checkpoint = torch.load(save_path)
    model.load_state_dict(checkpoint['model_state_dict'])  

_ = model.to(device)

found saved state at: model_resnet_best.pth


In [37]:
input_shapes = {
    (256, 256, 166) : 128,
}

labels = {
    0: "Cognitively Normal",
    1: "Alzheimer's Disease",
}

def read_image(path):
    input_img = nib.load(path).get_fdata()
    return input_img

def min_max_normalize(image, new_min=0, new_max=1):
    min_val = np.min(image)
    max_val = np.max(image)
    normalized_image = (image - min_val) / (max_val - min_val) * (new_max - new_min) + new_min
    return normalized_image

def separate_coronal_slices_around(img, slice_number, slices_count):
    slices = []
    for slice_no in range(slice_number - slices_count, slice_number + slices_count + 1):
        slice = np.rot90(img[:, slice_no, :], k=2)
        slice = min_max_normalize(slice)
        slices.append(slice) # coronal
    return slices

def predict(file, age, sex):
    img = read_image(file.name)
    if img.shape not in input_shapes:
        return "Incorrect file dimensions, try another"
    coronal_slices = separate_coronal_slices_around(img, input_shapes[img.shape], 0)
    coronal_slices = torch.tensor(coronal_slices, dtype=torch.float).to(device)

    x = coronal_slices.unsqueeze(0)
    y = torch.tensor((sex, age)).unsqueeze(0).to(device)
    print(y)
    print(x)
    # print(x.shape, y.shape)
    output = F.softmax(model(x, y), dim=1)
    # print(output)
    return labels[torch.argmax(output, dim=1).item()]


demo = gr.Interface(
    fn=predict,
    inputs=[
        gr.File(label="Upload .nii file"),
        gr.Slider(1, 100, step=1, label='Age'),
        gr.Radio(['Male', 'Female'], label='Sex', type='index')
    ],
    outputs='text',
    title="Alzheimer's Disease prediction"
)
demo.launch()

Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.14.0, however version 4.29.0 is available, please upgrade.
--------


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\stefa\AppData\Local\Programs\Python\Python311\Lib\shutil.py", line 825, in move
    os.rename(src, real_dst)
FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\stefa\\AppData\\Local\\Temp\\tmpr8_nur1b' -> 'C:\\Users\\stefa\\AppData\\Local\\Temp\\gradio\\66aa5b27a8fbc09f10e1e6372bf90789e8541ae3\\002_S_0295__2006-11-02.nii'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\stefa\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 408, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\stefa\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 84, in __call__
    return await self.app(scope, r

tensor([[ 0, 85]], device='cuda:0')
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0')
tensor([[ 0, 71]], device='cuda:0')
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0')
